In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller # 平稳性检验
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf # 画acf, pacf图
from statsmodels.stats.diagnostic import acorr_ljungbox # 白噪声检验
from statsmodels.graphics.api import qqplot # 画qq图
from sklearn.impute import KNNImputer
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error
import math
import itertools
matplotlib.rcParams['font.family'] = 'SimHei'
plt.rcParams['axes.unicode_minus'] = False

#读取数据
data = pd.read_csv(r'confirmdatahk.csv',index_col=0, engine='python')

#检查是否存在异常值
IQR = data['confirmedCount'].quantile(0.75) - data['confirmedCount'].quantile(0.25)
IQRL = data['confirmedCount'].quantile(0.25) - 3*IQR
IQRU = data['confirmedCount'].quantile(0.75) + 3*IQR
data["confirmedCount"] = data["confirmedCount"].mask((data["confirmedCount"] < IQRL) | (data["confirmedCount"] > IQRU), np.nan)

#检查是否存在缺失值
print(data.isna().sum())
imputer = KNNImputer(n_neighbors=4) #邻居样本求平均数
data=imputer.fit_transform(data)

#香港2022年COVID-19确诊数据可视化展示
data.plot(figsize=(15, 8), fontsize=12)
plt.xlabel('date', fontsize=12)
plt.ylabel('confirmedCount', fontsize=12)
plt.title("香港2022年COVID-19确诊人数")
plt.show()

#对确诊数据进行一阶差分，并进行可视化展示
data["confirmedCount_df"] = data["confirmedCount"].diff(1)
data["confirmedCount_df"].plot(figsize=(15, 8), fontsize=12)
plt.xlabel('date', fontsize=12)
plt.ylabel('diffconfirmedCount', fontsize=12)
plt.title("香港2022年COVID-19确诊人数差分数据-每日新增确诊")
plt.show()

#对差分数据进行adf平稳性检验
print(adfuller(data["confirmedCount_df"].dropna()))

#对差分数据进行LB白噪声检验
print(acorr_ljungbox(data["confirmedCount_df"].dropna()))

#画出差分后数据的自相关图
acf = plot_acf(data["confirmedCount_df"].dropna(), lags=40)
plt.title('差分后数据的自相关图')
acf.show()

#画出差分后数据的偏自相关图
pacf = plot_pacf(data["confirmedCount_df"].dropna(), lags=40)
plt.title('差分后数据的偏自相关图')
pacf.show()

#根据BIC准则选择模型阶数
p_min = 0
d_min = 0
q_min = 0
p_max = 5
d_max = 1
q_max = 5
results_bic = pd.DataFrame(index=['AR{}'.format(i) for i in range(p_min,p_max+1)],columns=['MA{}'.format(i) for i in range(q_min,q_max+1)])
for p,d,q in itertools.product(range(p_min,p_max+1),range(d_min,d_max+1),range(q_min,q_max+1)):
    if p==0 and d==0 and q==0:
        results_bic.loc['AR{}'.format(p), 'MA{}'.format(q)] = np.nan
        continue
    try:
        model = sm.tsa.ARIMA(data["confirmedCount_df"].dropna(), order=(p, d, q))
        results = model.fit()
        results_bic.loc['AR{}'.format(p), 'MA{}'.format(q)] = results.bic
    except:
        continue
results_bic = results_bic[results_bic.columns].astype(float)
fig, ax = plt.subplots(figsize=(10, 8))
ax = sns.heatmap(results_bic,mask=results_bic.isnull(),ax=ax,annot=True,fmt='.2f')
ax.set_title('BIC')
plt.show()


#模型拟合
model=sm.tsa.ARIMA(data["confirmedCount_df"].dropna(),order=(4,1,4)).fit()
model.summary()

#模型有效性检验
resid = model.resid
#对残差进行正态性检验
qqplot(resid, line='q', fit=True)
plt.show()
#对残差进行白噪声检验
print(acorr_ljungbox(resid))

#模型预测
predict_data = model.predict(1,360)
print('拟合数据')
print(predict_data)

forecast_data = model.forecast(6)
print('预测数据')
print(forecast_data)

fig = plt.subplots(1,1,figsize=(10,7),dpi=100)
plt.plot(predict_data, label='拟合数据')
plt.plot(forecast_data, label='预测数据')
plt.legend()
plt.title("ARIMA模型下香港2022年COVID-19每日新增人数拟合与预测图")
plt.show()

#计算RMSE
data = pd.read_csv(r'confirmdatahkb.csv',index_col=0, engine='python')
mse=mean_squared_error(data["confirmedCount"],data["predictCount"])
print('MSE: %.6f' % mse)
rmse = math.sqrt(mse)
print('RMSE: %.6f' % rmse)

data.plot(figsize=(15, 8), fontsize=12)
plt.xlabel('date', fontsize=12)
plt.ylabel('confirmedCount', fontsize=12)
plt.title("香港2022年COVID-19确诊人数真实值与预测值")
plt.show()